In [1]:
input_data = '/scratch/franzihe/input'
output_data = '/scratch/franzihe/output'
cs_in = '{}/CloudSat/'.format(input_data)
cs_out = '{}/CloudSat/'.format(output_data)
era_in = '{}/ERA5/monthly_means/0.25deg'.format(input_data)


In [2]:

import sys
import os
if os.path.isfile('/uio/kant/geo-metos-u1/franzihe/Documents/Python/globalsnow/eosc-nordic-climate-demonstrator/work/utils/imports.py') == True:
    sys.path.append('/uio/kant/geo-metos-u1/franzihe/Documents/Python/globalsnow/eosc-nordic-climate-demonstrator/work/utils')

from imports import(xr, ccrs, cy, plt, glob, fct, np, datetime, timedelta, h5py, da)


In [3]:
ff_cs = sorted(glob(cs_in+'2C-SNOW-PROFILE.P1_R05/2006/*/*.h5'))

In [4]:
GEOLOC_VARNAMES = ["DEM_elevation","Height", "Latitude", "Longitude"]
DATA_VARNAMES = ["snow_water_content", "snowfall_rate", "snowfall_rate_sfc"]
DATA_VARNAMES_EC = ["Pressure", "Temperature"]

In [5]:
datasets = []

for file in ff_cs[0:1]:
    year = int(file.split('/')[-1].split('_')[0][0:4])
    doy = int(file.split('/')[-1].split('_')[0][4:7])  # day of the year
    tt = datetime(year, 1, 1) + timedelta(doy - 1)
    
    
    file_ec = sorted(glob('{}ECMWF-AUX.P_R05/{}/{}/{}_*.h5'.format(cs_in, year, doy, file.split('/')[-1].split('_')[0])))[0]
    for month in range(6, 7):
        if tt.month == month:
            ds = xr.Dataset()
            
            h5file = h5py.File(file, "r")   # open h5 file
            ds['Profile_time'] = fct.get_profile_times(h5file['2C-SNOW-PROFILE'])

            for var in GEOLOC_VARNAMES:
                ds[var] = fct.get_geoloc_var(h5file['2C-SNOW-PROFILE'], var )

            for var in DATA_VARNAMES:
                ds[var] = fct.get_data_var(h5file['2C-SNOW-PROFILE'], var)

            # load ECMWF-AUX file
            h5file_ec = h5py.File(file_ec, "r")
            for var in DATA_VARNAMES_EC:
                ds[var] = fct.get_data_var(h5file_ec['ECMWF-AUX'], var)

            datasets.append(ds)

            # load era grid
            ds_era = xr.open_dataset('{}/clwc_Amon_ERA5_198501_198912.nc'.format(era_in, ))
            ds_era = ds_era.isel(time = month-1).squeeze().drop('clwc')

    ds_cs = xr.concat(datasets,dim='nray')

# add common pressure grid to dataset
ds_cs['Pressure_grid'] = xr.open_dataset('{}ECMWF-AUX.P_R05/pressure_grid.nc'.format(cs_out))['Pressure_grid']


DATA_VARNAMES = np.append(DATA_VARNAMES, DATA_VARNAMES_EC)

In [6]:
# convert ECMWF pressure to hPa
ds_cs['Pressure'] = ds_cs['Pressure']/100
ds_cs['Pressure'].attrs = {'units': ds_era['level'].attrs['units'], 'longname':ds_era['level'].attrs['long_name']}


In [10]:
# create dataset with same isobaric pressure levels
ds_pres = ds_cs.drop(('DEM_elevation', 'Height', ))
# create array

for var in DATA_VARNAMES:
    if (len(ds_cs[var].dims)) == 2:
        ds_pres[var] = xr.DataArray(data=da.full(shape = (ds_cs[var].shape), fill_value=np.nan, chunks=(len(ds_cs['nray'])/2, len(ds_cs['nbin']))), dims= list(ds_cs[var].dims), coords=list(ds_cs[var].indexes.values()))


        # Before performing the average within a grid cell we will bring the Profiles on a common pressure grid which we created in get_pressure_grid.py
        for nray in range(len(ds_cs.nray)):
            for nbin in range(len(ds_cs.nbin)):
                
                dmin= int(abs(ds_cs['Pressure'].isel(nray = nray) - ds_cs['Pressure_grid'].isel(nbin = nbin)).idxmin(dim = 'nbin'))
                ds_pres[var][nray, nbin] =  ds_cs[var].isel(nray = nray, nbin = dmin)

In [ ]:
lat_lon = xr.Dataset()

# create array 
for var in DATA_VARNAMES:
    if len(ds_pres[var].dims) == 1:
        lat_lon[var] = xr.DataArray(data=da.full(shape = (ds_era['longitude'].shape[0], ds_era['latitude'].shape[0], ), fill_value=np.nan, chunks=(ds_era['longitude'].shape[0]/2, ds_era['latitude'].shape[0])), 
                                                               dims=[ds_era['longitude'].dims[0], ds_era['latitude'].dims[0]], 
                                                               coords=[ds_era['longitude'].values, ds_era['latitude'].values])
    if len(ds_pres[var].dims) == 2:
        lat_lon[var] = xr.DataArray(data=da.full(shape = (ds_era['longitude'].shape[0], ds_era['latitude'].shape[0], ds_pres['Pressure_grid'].shape[0]), fill_value=np.nan, chunks=(ds_era['longitude'].shape[0]/2, ds_era['latitude'].shape[0], ds_pres['Pressure_grid'].shape[0])), 
                                                               dims=[ds_era['longitude'].dims[0], ds_era['latitude'].dims[0], ds_pres['Pressure_grid'].dims[0]], 
                                                               coords=[ds_era['longitude'].values, ds_era['latitude'].values, ds_pres['Pressure_grid'].values])

    # Calculate the average value within a grid box
    for i in range(len(lat_lon.longitude)+1):
        # loop through longitude
        if i == len(lat_lon.longitude):
            lon1 = 360
        else:
            lon1 = lat_lon['longitude'].isel(longitude=slice(i, i+2)).mean().values.tolist()
        if i == 0:
            lon0 = lat_lon['longitude'].isel(longitude=slice(i, i+1)).mean().values.tolist()
        else:
            lon0 = lat_lon['longitude'].isel(longitude=slice(i-1, i+1)).mean().values.tolist()

        # loop through latitude
        for k in range( len(lat_lon.latitude)):
            lat0 = lat_lon['latitude'].isel(latitude = slice(k, k+2)).mean().values.tolist()
            if k == 0:
                lat1 = lat_lon['latitude'].isel(latitude = slice(k, k+1)).mean().values.tolist()
            else:
                lat1 = lat_lon['latitude'].isel(latitude = slice(k-1, k+1)).mean().values.tolist()
            
            bbox = (lon0, lon1, lat0, lat1)
            # print(bbox)

            filter = ((ds_pres['Longitude'] > bbox[0]) & (ds_pres['Longitude'] < bbox[1]) & \
                (ds_pres['Latitude'] > bbox[2]) & (ds_pres['Latitude'] < bbox[3]))

            if len(ds_pres[var].dims) == 1:
                lat_lon[var][i, k] = ds_pres[var].where(filter).mean('nray',skipna=True,keep_attrs=True)
            if len(ds_pres[var].dims) == 2:
                lat_lon[var][i, k,:] = ds_pres[var].where(filter).mean('nray',skipna=True,keep_attrs=True)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111, projection=ccrs.Robinson())
ax.coastlines()
# ax.set_global()
# ax.set_extent([-30, 50, 50, 85], crs=ccrs.PlateCarree())
ax.set_extent([-10, 30, 25, 85], crs=ccrs.PlateCarree())

ax.plot(combined['Longitude'].where(filter), combined['Latitude'].where(filter), linewidth=2, transform=ccrs.PlateCarree());